# Example: convert ModelE3 SCM output to DEPHY format

Code to read ModelE3  output files and write to DEPHY format (NetCDF)

Contributed by Ann Fridlind from NASA/GISS

## Import libraries

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import netCDF4
import datetime as dt
from netCDF4 import Dataset

## Specify directory locations

If on the ARM JupyterHub, it is recommended to create and specify a local directory that is outside of the COMBLE-MIP repository to upload raw model output files in your model's format. 

Processed domain-mean outputs are invited for commit to the GitHub repository on a user-specified branch under /comble-mip/output_scm/YOUR_MODEL_NAME/sandbox/YOUR_OUTPUT_FILE_NAME. These can be committed and removed at any time.

If you are able to make a run without ice, it is requested to append 'noice' to YOUR_OUTPUT_FILE_NAME, so that it can readily be automatically compared with the baseline and other liquid-only runs.

In [2]:
# specify input and output file names: versions of ModelE3 with and without ice

# Phys
my_input_suffix = 'entppe.nc'
my_output_suffix = 'Phys_FixN.nc'

# Phys without ice
# my_input_suffix = 'entppe_noice.nc'
# my_output_suffix = 'Phys_FixN_noice.nc'

# Tun1
# my_input_suffix = 't698ml.nc'
# my_output_suffix = 'Tun1_FixN.nc'

# Tun1 without ice
# my_input_suffix = 't698ml_noice.nc'
# my_output_suffix = 'Tun1_FixN_noice.nc'

# Tun2
# my_input_suffix = 't705ml.nc'
# my_output_suffix = 'Tun2_FixN.nc'

# Tun2 without ice
# my_input_suffix = 't705ml_noice.nc'
# my_output_suffix = 'Tun2_FixN_noice.nc'

# specify local source directories (with subdirectories for spin up over ice and restart over water)
my_input_dir = '/data/home/fridlind/modelE3/'

# specify Github scratch directory where processed model output will be committed (automate later)
my_output_filename = 'ModelE3-' + my_output_suffix
my_gitdir = '../../output_scm/modelE/sandbox/'

## Read ModelE3 data

### Read single file containing all output data

Note: ERROR 1: PROJ... message can be ignored here.

In [3]:
input_filename = my_input_dir + '/allsteps.allmergeSCM_COMBLE-MIP_' + my_input_suffix
modele_data = xr.open_dataset(input_filename)

# check if the run contains ice variables
do_ice = bool(max(modele_data['iwp'].values)>0.)
print('do_ice = ',do_ice)

# full parameter list
modele_data

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


do_ice =  True


<xarray.Dataset>
Dimensions:          (lon: 1, lat: 1, time: 40, p: 110)
Coordinates:
  * lon              (lon) float32 10.0
  * lat              (lat) float32 74.5
  * time             (time) object 2020-03-12 22:15:00 ... 2020-03-13 17:45:00
  * p                (p) float32 979.0 969.0 959.0 949.0 ... 0.014 0.0075 0.0035
Data variables: (12/99)
    axyp             (lat, lon) float32 ...
    prsurf           (time, lat, lon) float32 ...
    gtempr           (time, lat, lon) float32 ...
    shflx            (time, lat, lon) float32 ...
    lhflx            (time, lat, lon) float32 ...
    ustar            (time, lat, lon) float32 ...
    ...               ...
    vm_mcci          (time, p, lat, lon) float32 ...
    vm_mcpi          (time, p, lat, lon) float32 ...
    dq_ls            (time, p, lat, lon) float32 ...
    dth_ls           (time, p, lat, lon) float32 ...
    dq_nudge         (time, p, lat, lon) float32 ...
    dth_nudge        (time, p, lat, lon) float32 ...
Attributes:
    xlabel:   SCM_COMBLE-MIP_entppe COMBLE-MIP Ent PPE (CAO case using Single...

### Calculate and append additional variables

In [4]:
dummy_sca = modele_data['lwp']*0.
modele_data = modele_data.assign(clwpt = dummy_sca + modele_data['cLWPss'].data + modele_data['cLWPmc'].data)
modele_data = modele_data.assign(rlwpt = dummy_sca + modele_data['pLWPss'].data + modele_data['pLWPmc'].data)
if do_ice: modele_data = modele_data.assign(iwpt = dummy_sca + modele_data['cIWPss'].data + modele_data['cIWPmc'].data + modele_data['pIWPss'].data + modele_data['pIWPmc'].data)
modele_data = modele_data.assign(tau = dummy_sca + modele_data['tau_ss'].data + modele_data['tau_mc'].data)

dummy_snd = modele_data['q']*0.
modele_data = modele_data.assign(rhobar = dummy_snd + 100.*modele_data['p_3d'].data/(287.05*modele_data['t'].data))
modele_data = modele_data.assign(qlc = dummy_snd + modele_data['qcl'].data + modele_data['QCLmc'].data)
modele_data = modele_data.assign(qlr = dummy_snd + modele_data['qpl'].data + modele_data['QPLmc'].data)
if do_ice: modele_data = modele_data.assign(qi = dummy_snd + modele_data['qci'].data + modele_data['qpi'].data + modele_data['QCImc'].data + modele_data['QPImc'].data)
modele_data = modele_data.assign(lcf = dummy_snd + modele_data['cldsscl'].data + modele_data['cldmccl'].data)
modele_data['lcf'].values = np.clip(modele_data['lcf'].values,0.,1.)
modele_data = modele_data.assign(prt = dummy_snd + modele_data['ssp_cl_3d'].data + modele_data['ssp_pl_3d'].data + modele_data['rain_mc'].data)
if do_ice: modele_data = modele_data.assign(pit = dummy_snd + modele_data['ssp_ci_3d'].data + modele_data['ssp_pi_3d'].data + modele_data['snow_mc'].data)
modele_data = modele_data.assign(dth_micro = dummy_snd + modele_data['dth_ss'].data + modele_data['dth_mc'].data)
modele_data = modele_data.assign(dq_micro = dummy_snd + modele_data['dq_ss'].data + modele_data['dq_mc'].data)

## Prepare output file in DEPHY format

### Read requested variables list

Variable description, naming, units, and dimensions.

In [5]:
# read list of requested variables
vars_mean_list = pd.read_excel('https://docs.google.com/spreadsheets/d/1Vl8jYGviet7EtXZuQiitrx4NSkV1x27aJAhxxjBb9zI/export?gid=1026157027&format=xlsx',
                              sheet_name='SCM')
pd.set_option('display.max_rows', None)
vars_mean_list

,standard_name,variable_id,units,dimensions,"comment (reported at end of each model physics time step, green=minimum, red=granularity enabling EMC2)"
0,time,time,s,–,"dimension, seconds since 2020-03-12 18:00:00"
1,pressure,pa,Pa,–,"dimension, pressure at model mid-level points"
2,surface_pressure,ps,Pa,time,–
3,surface_temperature,ts,K,time,–
4,surface_friction_velocity,ustar,m s-1,time,–
5,surface_upward_sensible_heat_flux,hfss,W m-2,time,–
6,surface_upward_latent_heat_flux,hfls,W m-2,time,–
7,obukhov_length,ol,m,time,–
8,pbl_height,pblh,m,time,PBL scheme layer thickness (if available)
9,inversion_height,zi,m,time,sharpest vertical gradient in air_potential_te...


### Match ModelE3 variables to requested outputs

Expand the table to include columns that indicate ModelE3 model variable names and any conversion factor.

In [6]:
# drop comments
vars_mean_list = vars_mean_list.drop(columns='comment (reported at end of each model physics time step, green=minimum, red=granularity enabling EMC2)')

# add columns to contain model output name and units conversion factors
vars_mean_list = vars_mean_list.assign(model_name='missing data',conv_factor=1.0)

In [7]:
# match to ModelE3 variable names and specify conversion factors
for index in vars_mean_list.index:
    standard_name = vars_mean_list.standard_name.iat[index]
    if standard_name=='surface_pressure': 
        vars_mean_list.model_name.iat[index] = 'prsurf'
        vars_mean_list.conv_factor.iat[index] = 100.
    if standard_name=='surface_temperature': 
        vars_mean_list.model_name.iat[index] = 'gtempr'
    if standard_name=='surface_friction_velocity': 
        vars_mean_list.model_name.iat[index] = 'ustar'
    if standard_name=='surface_upward_sensible_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'shflx'
    if standard_name=='surface_upward_latent_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'lhflx'
    if standard_name=='obukhov_length': 
        vars_mean_list.model_name.iat[index] = 'lmonin'
    if standard_name=='pbl_height': 
        vars_mean_list.model_name.iat[index] = 'pblht_bp'
    if standard_name=='inversion_height': 
        vars_mean_list.model_name.iat[index] = 'pblht_th'
    if standard_name=='atmosphere_mass_content_of_liquid_cloud_water': 
        vars_mean_list.model_name.iat[index] = 'clwpt'
    if standard_name=='atmosphere_mass_content_of_rain_water': 
        vars_mean_list.model_name.iat[index] = 'rlwpt'
    if do_ice:
        if standard_name=='atmosphere_mass_content_of_ice_water': 
            vars_mean_list.model_name.iat[index] = 'iwpt'
    if standard_name=='area_fraction_cover_of_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'cldtot_2d'
#    if standard_name=='area_fraction_cover_of_liquid_cloud': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='area_fraction_cover_of_convective_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'cldmc_2d'
    if standard_name=='optical_depth': 
        vars_mean_list.model_name.iat[index] = 'tau'
#    if standard_name=='optical_depth_of_liquid_water': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='precipitation_flux_at_surface': 
        vars_mean_list.model_name.iat[index] = 'prec'
    if standard_name=='precipitation_flux_of_ice_at_surface': 
        vars_mean_list.model_name.iat[index] = 'snowfall'
    if standard_name=='toa_outgoing_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'olr'
    if standard_name=='surface_downwelling_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'lwds'  
    if standard_name=='surface_upwelling_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'lwus'  
    if standard_name=='height': 
        vars_mean_list.model_name.iat[index] = 'z'
    if standard_name=='eastward_wind': 
        vars_mean_list.model_name.iat[index] = 'u'
    if standard_name=='northward_wind': 
        vars_mean_list.model_name.iat[index] = 'v'
    if standard_name=='air_dry_density': 
        vars_mean_list.model_name.iat[index] = 'rhobar'
    if standard_name=='air_temperature': 
        vars_mean_list.model_name.iat[index] = 't'
    if standard_name=='water_vapor_mixing_ratio': 
        vars_mean_list.model_name.iat[index] = 'q'
    if standard_name=='relative_humidity': 
        vars_mean_list.model_name.iat[index] = 'rhw'
    if standard_name=='relative_humidity_over_ice': 
        vars_mean_list.model_name.iat[index] = 'rhi'
    if standard_name=='air_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'th'
    if standard_name=='mass_mixing_ratio_of_cloud_liquid_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'qlc'
    if standard_name=='mass_mixing_ratio_of_rain_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'qlr'
    if do_ice: 
        if standard_name=='mass_mixing_ratio_of_ice_water_in_air': 
            vars_mean_list.model_name.iat[index] = 'qi'
    if standard_name=='area_fraction_of_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'cfr'
    if standard_name=='area_fraction_of_liquid_cloud': 
        vars_mean_list.model_name.iat[index] = 'lcf'
    if standard_name=='area_fraction_of_convective_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'cldmcr'
    if standard_name=='precipitation_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'prt'
    if do_ice:
        if standard_name=='precipitation_flux_in_air_in_ice_phase': 
            vars_mean_list.model_name.iat[index] = 'pit'
    if standard_name=='specific_turbulent_kinetic_energy': 
        vars_mean_list.model_name.iat[index] = 'e_turb'
    if standard_name=='disspation_rate_of_turbulent_kinetic_energy': 
        vars_mean_list.model_name.iat[index] = 'dissip_tke_turb'
#    if standard_name=='zonal_momentum_flux': 
#        vars_mean_list.model_name.iat[index] = ''
#    if standard_name=='meridional_momentum_flux': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='variance_of_upward_air_velocity': 
        vars_mean_list.model_name.iat[index] = 'w2_turb'
#    if standard_name=='vertical_flux_potential_temperature': 
#        vars_mean_list.model_name.iat[index] = ''
#    if standard_name=='vertical_flux_liquid_water_potential_temperature': 
#        vars_mean_list.model_name.iat[index] = ''
#    if standard_name=='vertical_flux_water_vapor': 
#        vars_mean_list.model_name.iat[index] = ''
#    if standard_name=='vertical_flux_total_water': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='convection_updraft_mass_flux': 
        vars_mean_list.model_name.iat[index] = 'lwdp'
    if standard_name=='convection_downdraft_mass_flux': 
        vars_mean_list.model_name.iat[index] = 'lwdp'
    if standard_name=='downwelling_longwave_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'lwdp'
    if standard_name=='upwelling_longwave_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'lwup'
    if standard_name=='tendency_of_air_potential_temperature_due_to_radiative_heating': 
        vars_mean_list.model_name.iat[index] = 'dth_lw'
    if standard_name=='tendency_of_air_potential_temperature_due_to_microphysics': 
        vars_mean_list.model_name.iat[index] = 'dth_micro'
    if standard_name=='tendency_of_air_potential_temperature_due_to_mixing': 
        vars_mean_list.model_name.iat[index] = 'dth_turb'
    if standard_name=='tendency_of_specific_humidity_due_to_microphysics': 
        vars_mean_list.model_name.iat[index] = 'dq_micro'
    if standard_name=='tendency_of_specific_humidity_due_to_mixing': 
        vars_mean_list.model_name.iat[index] = 'dq_turb'
    if standard_name=='mass_mixing_ratio_of_liquid_cloud_water_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'qcl'
    if standard_name=='mass_mixing_ratio_of_rain_water_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'qpl'
    if do_ice:
        if standard_name=='mass_mixing_ratio_of_ice_cloud_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'qci'
        if standard_name=='mass_mixing_ratio_of_ice_precipitation_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'qpi'
    if standard_name=='mass_mixing_ratio_of_liquid_cloud_water_in_air_convective': 
        vars_mean_list.model_name.iat[index] = 'QCLmc'
    if standard_name=='mass_mixing_ratio_of_rain_water_in_air_convective': 
        vars_mean_list.model_name.iat[index] = 'QPLmc'
    if do_ice:
        if standard_name=='mass_mixing_ratio_of_ice_cloud_in_air_convective': 
            vars_mean_list.model_name.iat[index] = 'QCImc'
        if standard_name=='mass_mixing_ratio_of_ice_precipitation_in_air_convective': 
            vars_mean_list.model_name.iat[index] = 'QPImc'
    if standard_name=='number_of_liquid_cloud_droplets_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'ncl'
    if standard_name=='number_of_rain_drops_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'npl'
    if do_ice:
        if standard_name=='number_of_ice_cloud_crystals_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'nci'
        if standard_name=='number_of_ice_precipitation_crystals_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'npi'
    if standard_name=='effective_radius_of_liquid_cloud_droplets_convective': 
        vars_mean_list.model_name.iat[index] = 're_mccl'
    if standard_name=='effective_radius_of_rain_convective': 
        vars_mean_list.model_name.iat[index] = 're_mcpl'
    if do_ice:
        if standard_name=='effective_radius_of_ice_cloud_convective': 
            vars_mean_list.model_name.iat[index] = 're_mcci'
        if standard_name=='effective_radius_of_ice_precipitation_convective': 
            vars_mean_list.model_name.iat[index] = 're_mcpi'
    if standard_name=='area_fraction_of_liquid_cloud_stratiform': 
        vars_mean_list.model_name.iat[index] = 'cldsscl'
    if standard_name=='area_fraction_of_rain_stratiform': 
        vars_mean_list.model_name.iat[index] = 'cldsspl'
    if do_ice:
        if standard_name=='area_fraction_of_ice_cloud_stratiform': 
            vars_mean_list.model_name.iat[index] = 'cldssci'
        if standard_name=='area_fraction_of_ice_precipitation_stratiform': 
            vars_mean_list.model_name.iat[index] = 'cldsspi'
    if standard_name=='area_fraction_of_liquid_cloud_convective': 
        vars_mean_list.model_name.iat[index] = 'cldmccl'
    if standard_name=='area_fraction_of_rain_convective': 
        vars_mean_list.model_name.iat[index] = 'cldmcpl'
    if do_ice:
        if standard_name=='area_fraction_of_ice_cloud_convective': 
            vars_mean_list.model_name.iat[index] = 'cldmcci'
        if standard_name=='area_fraction_of_ice_precipitation_convective': 
            vars_mean_list.model_name.iat[index] = 'cldmcpi'
    if standard_name=='mass_weighted_fall_speed_of_liquid_cloud_water_stratiform': 
        vars_mean_list.model_name.iat[index] = 'vm_sscl'
    if standard_name=='mass_weighted_fall_speed_of_rain_stratiform': 
        vars_mean_list.model_name.iat[index] = 'vm_sspl'
    if do_ice:
        if standard_name=='mass_weighted_fall_speed_of_ice_cloud_stratiform': 
            vars_mean_list.model_name.iat[index] = 'vm_ssci'
        if standard_name=='mass_weighted_fall_speed_of_ice_precipitation_stratiform': 
            vars_mean_list.model_name.iat[index] = 'vm_sspi'
    if standard_name=='mass_weighted_fall_speed_of_liquid_cloud_water_convective': 
        vars_mean_list.model_name.iat[index] = 'vm_mccl'
    if standard_name=='mass_weighted_fall_speed_of_rain_convective': 
        vars_mean_list.model_name.iat[index] = 'vm_mcpl'
    if do_ice:
        if standard_name=='mass_weighted_fall_speed_of_cloud_ice_crystals_convective': 
            vars_mean_list.model_name.iat[index] = 'vm_mcci'
        if standard_name=='mass_weighted_fall_speed_of_ice_precipitation_convective': 
            vars_mean_list.model_name.iat[index] = 'vm_mcpi'

vars_mean_list[2:] # echo variables (first two rows are dimensions)

,standard_name,variable_id,units,dimensions,model_name,conv_factor
2,surface_pressure,ps,Pa,time,prsurf,100.0
3,surface_temperature,ts,K,time,gtempr,1.0
4,surface_friction_velocity,ustar,m s-1,time,ustar,1.0
5,surface_upward_sensible_heat_flux,hfss,W m-2,time,shflx,1.0
6,surface_upward_latent_heat_flux,hfls,W m-2,time,lhflx,1.0
7,obukhov_length,ol,m,time,lmonin,1.0
8,pbl_height,pblh,m,time,pblht_bp,1.0
9,inversion_height,zi,m,time,pblht_th,1.0
10,atmosphere_mass_content_of_liquid_cloud_water,lwpc,kg m-2,time,clwpt,1.0
11,atmosphere_mass_content_of_rain_water,lwpr,kg m-2,time,rlwpt,1.0


## Create DEPHY output file

Write a single file to contain all domain-mean scalar and profile outputs. This code expects the write directory to be pre-existing (already created by the user). In the case that this output will be committed to the comble-mip GitHub repository, see above "Specify directory locations".

In [8]:
# create DEPHY output file
dephy_filename = './' + my_gitdir + my_output_filename
if os.path.exists(dephy_filename):
    os.remove(dephy_filename)
    print('The file ' + dephy_filename + ' has been deleted successfully')    
dephy_file = Dataset(dephy_filename,mode='w',format='NETCDF3_CLASSIC')
start_date = '2020-03-12T22:00:00Z'

# create global attributes
dephy_file.title='ModelE3 SCM results for COMBLE-MIP case: fixed stratiform Nd and Ni'
dephy_file.reference='https://github.com/ARM-Development/comble-mip'
dephy_file.authors='Ann Fridlind (ann.fridlind@nasa.gov), Florian Tornow (florian.tornow@nasa.gov), Andrew Ackerman (andrew.ackerman@nasa.gov)'
dephy_file.source=input_filename
dephy_file.version=dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dephy_file.format_version='DEPHY SCM format version 1.6'
dephy_file.script='convert_ModelE3_SCM_output_to_dephy_format.ipynb'
dephy_file.startDate=start_date
dephy_file.force_geo=1
dephy_file.surfaceType='ocean'
dephy_file.surfaceForcing='ts'
dephy_file.lat='74.5 deg N'
dephy_file.dp='see pressure variable'
dephy_file.np=modele_data.sizes['p']

# create dimensions
nt = modele_data.dims['time']
time = dephy_file.createDimension('time', nt)
time = dephy_file.createVariable('time', np.float64, ('time',))
time.units = 'seconds since ' + dephy_file.startDate
time.long_name = 'time'
# find time step and build time in seconds
time1 = dt.datetime.strptime(str(modele_data['time'].data[0]),'%Y-%m-%d %H:%M:%S')
time2 = dt.datetime.strptime(str(modele_data['time'].data[1]),'%Y-%m-%d %H:%M:%S')
delta_t = (time2-time1).total_seconds()
time[:] = (np.arange(nt)+1.)*delta_t

nl = modele_data.dims['p']
pa = dephy_file.createDimension('pa', nl)
pa = dephy_file.createVariable('pa', np.float64, ('pa',))
pa.units = 'Pa'
pa.long_name = 'pressure'
pa[:] = modele_data['p'].data

# create and fill variables
for index in vars_mean_list.index[2:]:
    std_name = vars_mean_list.standard_name.iat[index]
#   print(std_name) # debug
    var_name = vars_mean_list.variable_id.iat[index]
    mod_name = vars_mean_list.model_name.iat[index]
    c_factor = vars_mean_list.conv_factor.iat[index]
    if vars_mean_list.dimensions.iat[index]=='time':
        new_sca = dephy_file.createVariable(var_name, np.float64, ('time'))
        new_sca.units = vars_mean_list.units.iat[index]
        new_sca.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data':
            new_sca[:] = modele_data[mod_name].data*c_factor
    if vars_mean_list.dimensions.iat[index]=='time, pressure':
        new_snd = dephy_file.createVariable(var_name, np.float64, ('time','pa'))
        new_snd.units = vars_mean_list.units.iat[index]
        new_snd.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data': 
            new_snd[:] = modele_data[mod_name].data*c_factor

print(dephy_file)
dephy_file.close()

The file ./../../output_scm/modelE/sandbox/ModelE3-Phys_FixN.nc has been deleted successfully
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: ModelE3 SCM results for COMBLE-MIP case: fixed stratiform Nd and Ni
    reference: https://github.com/ARM-Development/comble-mip
    authors: Ann Fridlind (ann.fridlind@nasa.gov), Florian Tornow (florian.tornow@nasa.gov), Andrew Ackerman (andrew.ackerman@nasa.gov)
    source: /data/home/fridlind/modelE3//allsteps.allmergeSCM_COMBLE-MIP_entppe.nc
    version: 2023-09-25 22:23:42
    format_version: DEPHY SCM format version 1.6
    script: convert_ModelE3_SCM_output_to_dephy_format.ipynb
    startDate: 2020-03-12T22:00:00Z
    force_geo: 1
    surfaceType: ocean
    surfaceForcing: ts
    lat: 74.5 deg N
    dp: see pressure variable
    np: 110
    dimensions(sizes): time(40), pa(110)
    variables(dimensions): float64 time(time), float64 pa(pa), float64 ps(time), float64 ts(time), float6

### Check output file

In [9]:
dephy_check = xr.open_dataset(dephy_filename)
dephy_check

<xarray.Dataset>
Dimensions:    (time: 40, pa: 110)
Coordinates:
  * time       (time) datetime64[ns] 2020-03-12T22:30:00 ... 2020-03-13T18:00:00
  * pa         (pa) float64 979.0 969.0 959.0 949.0 ... 0.014 0.0075 0.0035
Data variables: (12/91)
    ps         (time) float64 ...
    ts         (time) float64 ...
    ustar      (time) float64 ...
    hfss       (time) float64 ...
    hfls       (time) float64 ...
    ol         (time) float64 ...
    ...         ...
    vmics      (time, pa) float64 ...
    vmips      (time, pa) float64 ...
    vmlcc      (time, pa) float64 ...
    vmlrc      (time, pa) float64 ...
    vmicc      (time, pa) float64 ...
    vmipc      (time, pa) float64 ...
Attributes: (12/14)
    title:           ModelE3 SCM results for COMBLE-MIP case: fixed stratifor...
    reference:       https://github.com/ARM-Development/comble-mip
    authors:         Ann Fridlind (ann.fridlind@nasa.gov), Florian Tornow (fl...
    source:          /data/home/fridlind/modelE3//allsteps.allmergeSCM_COMBLE...
    version:         2023-09-25 22:23:42
    format_version:  DEPHY SCM format version 1.6
    ...              ...
    force_geo:       1
    surfaceType:     ocean
    surfaceForcing:  ts
    lat:             74.5 deg N
    dp:              see pressure variable
    np:              110